# Ce programme ne fonctionne qu'avec la partie émission

In [1]:
#------------- Importation des modules -----------------


import pyaudio   #Importion du module PyAudio (émettre le son)
import wave      #Importion du module wave (lire, telecharger ou jouer un fichier Wav)
import numpy as np #Importion du module numpy (taleau à deux dimensions, PI, fonction maths (sin, cos, etc...))
import matplotlib.pyplot as plt #Importion du module matplotlib.pyplot (Graphique)
import binascii   #Importion du module binascii (conversion binaire / ascii, )
from os.path import abspath   #Importion du module abspath permettant de donner l'endroit ou se trouve un fichire (quand il est téléchargé par exemple)
import time  #Importion du module(créer des pauses dans le programme)
import soundfile as sf      #Importion du module soundfile permettant de traiter des fichiers sons
import sounddevice as sd    #Importion du module sounddevice fournissant des fonctions pour lire et enregistrer des tableaux numpy contenant des signaux audios
import scipy.io.wavfile #Importation tu module scipy.io permettant d'obtenir de nombreux modules, classes et fonctions disponibles pour lire et écrire des données dans une variété de formats de fichiers
import scipy.signal as sig  #Importation du module scipy.signal permettant de filtrer une séquence de données à l'aide d'un filtre numérique
from scipy.signal import butter, filtfilt, lfilter #Importation des modules butter, filtfilt et lfilter du module scipy.signal (nous avons rajouté cette ligne car en l'enlevant, le programme ne fonctionnait plus, dû à un bug)
from functools import partial #Importation du module partials de functools permettant d'appeler un objet partiel (ce sont des objets appelables créees par le module partial)


# --------- Création de la fonction décodage --------


def decodeData(data_crc, Clé):   #Création de la fonction decodeData qui permettra de recomparer les clés CRC afin de détecter si il ya eu une erreur 
   
    l_Clé = len(Clé)  #La variable l_clé prend en valeur la taille du tableau clé
    appended_data = data_crc + '0'*(l_Clé-1) 
    remainder = mod2div(appended_data, Clé) 
   
    return remainder 



# ------------ Démodulation -----------------

  
T = np.arange(0.0, 2*N)/Fe  #Création d'un vecteur temps servant pour les porteuses afin de démoduler le message

Fp1 = 23000     #Fréquence de la porteuse 1
Fp2 = 21000     #Fréquence de la porteuse 2

S1 = A1*np.sin(2*np.pi*Fp1*T)  #Utilisation de la même formule des porteuses que pour envoyer le message (seul le vecteur temps change)
S2 = A2*np.sin(2*np.pi*Fp2*T)  #Utilisation de la même formule des porteuses que pour envoyer le message (seul le vecteur temps change)

bit1 = Msg_FSK*S1   #La variable bit1 prend en valeur le produit de Msg_FSK et S1
bit0 = Msg_FSK*S2   #La variable bit0 prend en valeur le produit de Msg_FSK et S2
   
y1 = []  #Création d'un tableau y1
y2 = []  #Création d'un tableau y2

z=0
for z in range(0, 2*N, Ns): 
    y1.append(np.trapz(bit1[z:z+Ns],T[z:z+Ns]))  #Dans la dernière case du tableau de y1, le programme ajoute la formule ci contre 
    y2.append(np.trapz(bit0[z:z+Ns],T[z:z+Ns]))  #Dans la dernière case du tableau de y2, le programme ajoute la formule ci contre 

comparateur = (np.array(y1) > np.array(y2))  #Création d'un comparateur qui compare chaque valeur du tableau de y1 par rapport à y2


# ------------ Liste pour stocker le message démodulé -----------------

Message_démodule = []  #Création d'un tableau 

for ii in range (0, len(comparateur)):
    
    if comparateur [ii] == False:   #Si la case du tableau est négative alors la dernière case du tableau prend 0
        Message_démodule.extend([int(0)])
        
    if comparateur [ii] == True:
         Message_démodule.extend([int(1)]) #Si la case du tableau est positive alors la dernière case du tableau prend 1
        
             
print("Le message binaire codé est:\n ", Liste_2bin)
print("Le message binaire démodulé est:\n ", Message_démodule)

# --------------- Calculer si erreur de réception -------------------


Erreur = []
Erreur = (np.array(Liste_2bin) == np.array(Message_démodule))  #Comparaison de la liste du message codé et du message démodulé

print ("Système de détection d'erreur :\n", Erreur) #Affichage des erreurs (True = aucune erreur et False = erreur)

       
# -------------- Décodage du message -----------------


Message_decode = [] #Création d'un tableau


for w in range (0, len(Message_démodule), 2 ):#Fonction for de 0 jusque la fin du message démodulé avec un pas de 2 puisqu'on utilise un décodage synchrone Manchester
    Message_decode.extend(str(Message_démodule[w])) #La dernière valeur du tableau prend une valeur sur 2 du message démodulé afin de respecter le codage Manchester
    
Message_decode = list(map(int, Message_decode))    #Convertion de la liste de string en entier

check = decodeData(str(Message_decode), Clé) #La variable check prend la valeur de la fonction decodedata avec en paramètre Message_decode et la clé CRC


# -------- Conversion binaire ASCII ---------


bin_data = ""  #Création d'une variable pouvant contenir des caractères
for elem in Message_decode:   #Pour chaque élement dans le tableau message_decode
    bin_data += str(elem)  #Bin_data prend la valeur du tableau en string
    
data_reçu =' ' #Création d'une variable pouvant contenir des caractères

def BinaryToDecimal(binary):  #Création d'une fonction pouvant convertir le mesasge binaire en décimal
    binary1 = binary #La variable binary1 prend la valeur binary
    decimal, i, n = 0, 0, 0 #initialisation des variables
    
    while(binary != 0): #Tant que binary n'est pas égal à 0
        dec = binary % 10  #La variable dec prend binary % 10
        decimal = decimal + dec * pow(2, i) 
        binary = binary//10
        i += 1
    return (decimal) #Retourne la valeur finale

for i in range(0, len(bin_data), 8): 
    temp_data = int(bin_data[i+1:i+8]) #Utilisation de temp_data pour obtenir la valeur décimale correspondante de (temp_data)
      
    decimal_data = BinaryToDecimal(temp_data) #La variable decimal_data prend la BinaryToDecimal valeur de temp_data après être passé dans la fonction

    

    # Décodage de la valeur décimale renvoyée par la fonction BinarytoDecimal en utilisant char(decimal_data)
    data_reçu = data_reçu + chr(decimal_data) 


print("Le message reçu est :", data_reçu) #Affichage du message reçu, démodulé, décodé et reconverti en ASCII


print("Le reste de la division après décodage est ->" + check)  #Affichage de check, du reste de la division après le décodage
temp = "0" * (len(Clé) - 1) #Création d'une variable temps pour vérifier la valeur CRC


if check == temp: #Si la variable check a la même valeur que la variable temp
    print("Les données -> "+ Msg_bin + " <- sont reçues !") #Affichage de la vérifiaction CRC.

NameError: name 'N' is not defined